In [71]:
import ray
from collections import Counter
import time


print("Testrun")

ds = ray.data.read_csv("../testset.csv")
print("Input read: ", ds)

"""
Function, welche die Purchases per Customer zählt
"""
@ray.remote
def count_customers(s):
    counter = Counter()
    for record in s.iter_rows():
        counter.update({record["CustomerId"]: 1})
    """
    dictionary = {}

    for record in s.iter_rows():
        if record["CustomerId"] in dictionary:
            dictionary[record["CustomerId"]] = dictionary.get(record["CustomerId"]) + 1
        else:
            dictionary[record["CustomerId"]] = 1
    
    counter = Counter(dictionary)
    """
    return counter

"""Function, welche Counter addiert"""
@ray.remote
def add_counters(fst, snd):
    counter1 = Counter(fst)
    counter2 = Counter(snd)
    return counter1 + counter2


"""
Function, welche die Top 10 Customer findet
"""
@ray.remote
def find_top_10_customers(customers):
    counter = Counter(customers)
    return counter.most_common(10)

start = time.time()

print("Splitting")
split_indices = []
for i in range(1,5):
    split_indices.append(i*200000)
    
    
print(split_indices)
    
splits = ds.split_at_indices(split_indices)
print("Functioning")
customer_lists = []
for split in splits:
    x = count_customers.remote(split)
    #print(ray.get(x))
    customer_lists.append(x)

while len(customer_lists) > 1:
    customer_lists = customer_lists[2:] + [add_counters.remote(customer_lists[0], customer_lists[1])]

#print(ray.get(customers))
ys = find_top_10_customers.remote(customer_lists[0])
print("Getting")
ys = ray.get(ys)

print(ys)


print(ds.count())

print('duration: ',time.time() - start)

Testrun
Input read:  Dataset(num_blocks=1, num_rows=None, schema={CustomerId: int64, OrderId: int64, ProductId: int64, ProductGroupId: int64, Quantity: double, OrderDate: date32[day]})
Splitting
[200000, 400000, 600000, 800000]
Functioning
Getting
[(15, 10224), (55, 8973), (14, 8776), (16, 8660), (69, 7741), (5659, 7089), (12788, 6940), (10841, 6755), (12835, 6320), (56, 5919)]
999999
duration:  27.223820209503174
